# Preliminary Analysis

In [1]:
import csv
import pandas
import spacy
import numpy
from collections import Counter
from py4j.java_gateway import JavaGateway

In [2]:
nlp = spacy.load('en')
gateway = JavaGateway()
metamap = gateway.entry_point

In [3]:
combined = pandas.read_csv('05 Combined Cleaned.csv', delimiter=',', encoding='latin-1').fillna('')
combined[:1]

,ID,Process Step,Problem Type,Contributing Factors,Overall Severity,Incident Description,Language,Translated,Neat Cleaned,Bony Cleaned
0,2511,Treatment delivery,"Wrong, missing, mislabeled, or damaged treatme...",Distraction or diversions involving staff,None,Non prescribed bolus . Bolus was not prescribe...,En,Non prescribed bolus . Bolus was not prescribe...,non prescribed bolus . bolus was not prescribe...,non prescribed bolus bolus prescribe md pt rec...


### Process Step

In [4]:
def data_element_composition(dataframe, column):
    composition = (dataframe.groupby(column)[dataframe.columns[0]].count()/len(dataframe)*100).round(2).sort_values(ascending=False)
    return composition

In [5]:
data_element_composition(combined, 'Process Step')

Process Step
Treatment delivery                                       34.10
                                                         20.28
Contouring and planning                                  18.54
Imaging for treatment planning                           13.39
Patient medical consultation and physician assessment     6.33
Pre-treatment quality assurance                           4.22
Radiation treatment prescription scheduling               1.48
On-treatment quality assurance                            1.06
Interventional procedure for planning and/or delivery     0.33
Post-treatment completion                                 0.26
Name: ID, dtype: float64

In [6]:
nm_ps = combined[combined['Process Step']!=''] 

In [7]:
set(nm_ps['Process Step'])

{'Contouring and planning',
 'Imaging for treatment planning',
 'Interventional procedure for planning and/or delivery',
 'On-treatment quality assurance',
 'Patient medical consultation and physician assessment',
 'Post-treatment completion',
 'Pre-treatment quality assurance',
 'Radiation treatment prescription scheduling',
 'Treatment delivery'}

In [8]:
nm_ps = combined[combined['Process Step']!=''] # nm means no missing
nm_ps.to_csv('06 NM PS.csv', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)

### Problem Type

In [9]:
data_element_composition(combined, 'Problem Type')

Problem Type
                                                                  39.05
Wrong patient position, setup point, or shift                     15.77
Radiation therapy scheduling error                                10.95
Wrong, missing, mislabeled, or damaged treatment accessories       9.76
Excess imaging dose                                                3.36
Systematic hardware/software (including dose-volume) error         3.10
Wrong target or OAR contours or wrong planning (Retired Value)     2.51
Wrong prescription dose-fractionation or calculation error         1.91
Failure to perform on-treatment imaging as per instructions        1.65
Inadequate coordination of combined modality care                  1.39
Wrong anatomical site (excluding laterality)                       1.39
Wrong patient                                                      1.32
Wrong side (laterality)                                            1.29
Fall or other patient injury or medical condition  

In [10]:
nm_pt = combined[combined['Problem Type']!='']
nm_pt.to_csv('06 NM PT.csv', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)

### Contributing Factors

In [11]:
cf_list = []
for cfs in combined['Contributing Factors']:
    cf_list += cfs.split('|')
cf_df = pandas.DataFrame(cf_list, columns=['Contributing Factors'])
data_element_composition(cf_df, 'Contributing Factors')

Contributing Factors
Policies and/or procedures not followed                                                       19.69
Distraction or diversions involving staff                                                     14.47
Expectation bias involving staff                                                              13.17
Communication or documentation inadequate (patient specific)                                  12.42
Staff behaviour                                                                                9.63
Policies and/or procedures non-existent or inadequate                                          8.09
                                                                                               7.17
Equipment software or hardware design, including 'human factors' design, inadequate            3.89
Failure to identify potential risks                                                            3.83
Patient or family member medical condition, preference or behaviour            

In [12]:
nm_cf = combined[combined['Contributing Factors']!=''] # dropna means dropping not available data
nm_cf.to_csv('06 NM CF.csv', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)

### Contributing Factors

In [13]:
data_element_composition(combined, 'Overall Severity')

Overall Severity
            46.90
None        43.54
Mild         8.67
Moderate     0.76
Severe       0.13
Name: ID, dtype: float64

In [14]:
nm_pt = combined[combined['Overall Severity']!='']
nm_pt.to_csv('06 NM OS.csv', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)

### Incident Description

##### 1-gram frequency

In [15]:
one_grams = []
for entry in combined['Bony Cleaned']:
    one_grams += set(entry.split())
one_gram_counter = Counter(one_grams)
', '.join([item[0] + ' ' + str(item[1]) for item in list(one_gram_counter.most_common())])

'treatment 944, date 886, patient 859, pt 842, cardinal 730, image 546, plan 531, time 433, treat 412, note 370, ct 367, take 366, incorrect 365, setup 345, s 304, qa 279, ro 278, ordinal 272, sim 267, field 261, cbct 257, scan 241, indicate 238, task 237, shift 222, send 221, start 221, pre 221, notice 218, set 216, document 213, planning 211, catch 205, no 200, book 198, documentation 196, the 195, prior 195, fraction 194, schedule 194, miss 192, complete 187, quantity 185, tx 183, unit 180, imaging 179, rt 178, require 176, error 171, call 165, not 165, instead 162, change 161, wrong 159, apron 158, check 149, physics 148, review 147, chart 145, bolus 141, correct 141, kv 141, deliver 139, come 139, tattoo 138, therapist 135, follow 134, planner 132, discover 129, new 128, xrt 128, post 123, day 121, need 118, match 117, breast 115, on 115, result 115, approve 114, dose 113, prescription 111, give 111, tmt 110, staff 109, delay 108, appointment 106, hold 105, be 100, radiation 98, s

In [16]:
two_grams = []
for entry in combined['Bony Cleaned']:
    words = entry.split()
    two_grams += set(zip(words, words[1:]))
two_gram_counter = Counter(two_grams)
', '.join([item[0][0] + ' ' + item[0][1] + ' ' + str(item[1]) for item in list(two_gram_counter.most_common())])

'ct sim 207, image take 180, setup note 151, patient s 108, cardinal fraction 96, date treatment 94, task send 88, pre tmt 77, catch pre 74, plan cardinal 73, cardinal cardinal 70, kv image 70, pt s 70, patient treat 68, start date 67, pt treat 67, patient setup 66, pre tx 65, date image 60, pt book 57, breath hold 57, tx qa 57, pre treatment 56, cardinal treatment 55, treatment unit 55, treatment date 54, date date 51, extra image 51, treatment plan 50, Ã¢ Ã£ 49, Ã£ Ã¢ 49, rad onc 46, treat date 46, ct scan 45, set note 45, treatment deliver 44, the patient 43, treatment field 43, sim setup 43, tmt qa 42, date pt 41, date qa 40, on date 39, catch date 38, physics qa 37, s u 37, prior treatment 37, image match 37, setup sheet 36, pt schedule 36, apron notice 36, cbct take 35, pt come 34, at time 34, incorrect tattoo 34, plan ready 33, ordinal fraction 33, isocentre shift 33, qa complete 32, pt scan 32, mri consent 32, fraction cardinal 31, onc req 31, cardinal plan 31, start treatment 

In [17]:
three_grams = []
for entry in combined['Bony Cleaned']:
    words = entry.split()
    three_grams += set(zip(zip(words, words[1:]), words[2:]))
three_gram_counter = Counter(three_grams)
', '.join([item[0][0][0] + ' ' + item[0][0][1] + ' ' + item[0][1] + ' ' + str(item[1]) for item in list(three_gram_counter.most_common())])

'catch pre tmt 51, pre tx qa 51, Ã¢ Ã£ Ã¢ 49, Ã£ Ã¢ Ã£ 49, extra image take 47, patient setup note 45, pre tmt qa 42, ct sim setup 38, rad onc req 31, sim setup sheet 29, mri consent form 26, pre tmt imaging 24, pre treatment imaging 24, Ã£ Ã¢ s 23, miss setup note 23, kv image take 21, plan setup note 20, perform catch pre 19, document patient setup 19, shift perform catch 18, setup note catch 18, catch pre treatment 18, isocentre shift perform 17, time ct sim 17, qa apron notice 17, kv image x 17, physics qa complete 16, ct sim set 16, pt hold breath 16, breath hold image 16, sim set sheet 15, hold image take 15, indicate ct sim 14, date chart qa 14, miss patient setup 14, pre treatment qa 14, consent form pt 13, incorrect isocentre shift 13, take pt hold 13, incorrect tattoo documentation 13, patient s treatment 12, pt treat date 12, sim ct sim 12, cardinal cardinal cardinal 12, info mri consent 12, image take date 12, treatment setup note 12, s u note 12, note catch date 12, select

Optimal thresholds for n-gram inclusion

|     | TF-IDF | Topic modeling |
|-----|--------|----------------|
| Max | 1000   | 600            |
| Min | 5      | 5              |

##### POS tagging

In [18]:
parts_of_speech = ['ADJ', 'ADV', 'NOUN', 'NUM', 'VERB']

In [19]:
columns = ['ADJ', 'ADV', 'NOUN', 'NUM', 'VERB', 'WC', 'SC']

In [20]:
def parts_of_speech_string(string):
    
    # Initialize row
    row = pandas.DataFrame(columns=columns).astype('int32')
    row.loc[0] = [0 for i in range(len(columns))]
    
    # POS count
    doc = nlp(string)
    for token in doc:
        try:
            row.loc[0][token.pos_] += 1
        except Exception:
            pass

    # Word and sentence count
    row.loc[0]['WC'] = int(row[parts_of_speech].sum(axis=1).loc[0])
    row.loc[0]['SC'] = len(list(doc.sents))
    return [float(element) for element in row.loc[0]]

In [21]:
def parts_of_speech_column(column):
    counts = pandas.DataFrame(columns=columns)
    for i in range(len(column)):
        string = column.iloc[i]
        counts.loc[i] = parts_of_speech_string(string)
    return counts

In [22]:
pos_counts = parts_of_speech_column(combined['Neat Cleaned'])
pos_counts[:5]

,ADJ,ADV,NOUN,NUM,VERB,WC,SC
0,3.0,1.0,6.0,0.0,3.0,13.0,3.0
1,5.0,2.0,25.0,0.0,15.0,47.0,6.0
2,1.0,3.0,7.0,0.0,6.0,17.0,1.0
3,2.0,1.0,7.0,0.0,2.0,12.0,2.0
4,0.0,1.0,7.0,0.0,2.0,10.0,3.0


In [23]:
def summary(dataframe):
    return dataframe.describe().round(2)[1:]

In [24]:
summary = summary(pos_counts)
summary

,ADJ,ADV,NOUN,NUM,VERB,WC,SC
mean,2.94,1.82,11.14,0.18,6.92,23.00,3.07
std,3.86,2.44,13.11,1.00,8.78,26.82,3.10
min,0.00,0.00,0.00,0.00,0.00,1.00,1.00
25%,1.00,0.00,4.00,0.00,2.00,9.00,1.00
50%,2.00,1.00,7.00,0.00,4.00,15.00,2.00
75%,4.00,2.00,13.00,0.00,8.00,27.00,4.00
max,53.00,26.00,161.00,32.00,98.00,294.00,38.00


In [25]:
summary.to_csv('06 POS Summary.csv', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)